In [1]:
import pandas as pd

In [2]:
import sys, os

sys.path.append(os.path.abspath(os.path.join("../..")))
sys.path.append(os.path.abspath(os.path.join("../scripts")))

In [3]:
import eda_functions as eda

In [4]:
import preprocess_functions as prep

In [5]:
import dvc.api

# Read Data

In [6]:
#path = 'data/train.csv'
#repo = '../'
#version = 'v1'

#data_url = dvc.api.get_url(path=path, repo=repo, rev=version)
train_data = pd.read_csv('../data/train.csv', low_memory= False)

In [7]:
#path = 'data/test.csv'
#repo = '../'
#version = 'v1'

#data_url = dvc.api.get_url(path=path, repo=repo, rev=version)
test_data = pd.read_csv('../data/test.csv',low_memory= False)

# Summary About Data

## General 

In [8]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1017209 entries, 0 to 1017208
Data columns (total 19 columns):
 #   Column                     Non-Null Count    Dtype  
---  ------                     --------------    -----  
 0   Date                       1017209 non-null  object 
 1   Store                      1017209 non-null  int64  
 2   DayOfWeek                  1017209 non-null  int64  
 3   Sales                      1017209 non-null  int64  
 4   Customers                  1017209 non-null  int64  
 5   Open                       1017209 non-null  int64  
 6   Promo                      1017209 non-null  int64  
 7   StateHoliday               1017209 non-null  object 
 8   SchoolHoliday              1017209 non-null  int64  
 9   StoreType                  1017209 non-null  object 
 10  Assortment                 1017209 non-null  object 
 11  CompetitionDistance        1014567 non-null  float64
 12  CompetitionOpenSinceMonth  693861 non-null   float64
 13  CompetitionO

In [9]:
test_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41088 entries, 0 to 41087
Data columns (total 18 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Date                       41088 non-null  object 
 1   Id                         41088 non-null  int64  
 2   Store                      41088 non-null  int64  
 3   DayOfWeek                  41088 non-null  int64  
 4   Open                       41077 non-null  float64
 5   Promo                      41088 non-null  int64  
 6   StateHoliday               41088 non-null  object 
 7   SchoolHoliday              41088 non-null  int64  
 8   StoreType                  41088 non-null  object 
 9   Assortment                 41088 non-null  object 
 10  CompetitionDistance        40992 non-null  float64
 11  CompetitionOpenSinceMonth  25872 non-null  float64
 12  CompetitionOpenSinceYear   25872 non-null  float64
 13  Promo2                     41088 non-null  int

In [18]:
# set Date column as datetime index
#train_data['Date'] = pd.to_datetime(train_data['Date'])
#train_data.set_index('Date', inplace=True)

#test_data['Date'] = pd.to_datetime(test_data['Date'])
#test_data.set_index('Date', inplace= True)

In [10]:
# set Date column as datetime index

train_data = prep.index_by_date(train_data, 'Date')
test_data = prep.index_by_date(test_data, 'Date')

## Columns with Missing Values

In [11]:
eda.missing_values_table(train_data)

Your selected dataframe has 18 columns.
There are 6 columns that have missing values.


,Missing Values,% of Total Values,Dtype
Promo2SinceWeek,508031,49.90,float64
Promo2SinceYear,508031,49.90,float64
PromoInterval,508031,49.90,object
CompetitionOpenSinceMonth,323348,31.80,float64
CompetitionOpenSinceYear,323348,31.80,float64
CompetitionDistance,2642,0.30,float64


In [12]:
eda.missing_values_table(test_data)

Your selected dataframe has 17 columns.
There are 7 columns that have missing values.


,Missing Values,% of Total Values,Dtype
Promo2SinceWeek,17232,41.90,float64
Promo2SinceYear,17232,41.90,float64
PromoInterval,17232,41.90,object
CompetitionOpenSinceMonth,15216,37.00,float64
CompetitionOpenSinceYear,15216,37.00,float64
CompetitionDistance,96,0.20,float64
Open,11,0.00,float64


# Add More Columns

2.1 Preprocessing
It is important to process the data into a format where it can be fed to a machine learning model. This typically means converting all non-numeric columns to numeric, handling NaN values and generating new features from already existing features. 

In our case, you have a few datetime columns to preprocess. you can extract the following from them:
- weekdays
- weekends 
- number of days to holidays
- Number of days after holiday
- Beginning of month, mid month and ending of month
- (think of more features to extract), extra marks for it
			
As a final thing, you have to scale the data. This helps with predictions especially when using machine learning algorithms that use Euclidean distances. you can use the standard scaler in sklearn for this.


In [13]:
train_data = prep.date_componants(train_data, train_data.index)

In [14]:
test_data = prep.date_componants(test_data, test_data.index)

In [15]:
train_data.columns

Index(['Store', 'DayOfWeek', 'Sales', 'Customers', 'Open', 'Promo',
       'StateHoliday', 'SchoolHoliday', 'StoreType', 'Assortment',
       'CompetitionDistance', 'CompetitionOpenSinceMonth',
       'CompetitionOpenSinceYear', 'Promo2', 'Promo2SinceWeek',
       'Promo2SinceYear', 'PromoInterval', 'IsWeekend', 'Year', 'Month',
       'DayOfMonth', 'BeginMonth', 'MidMonth', 'EndMonth', 'DayOfYear',
       'IsWeekday'],
      dtype='object')

In [16]:
test_data.columns

Index(['Id', 'Store', 'DayOfWeek', 'Open', 'Promo', 'StateHoliday',
       'SchoolHoliday', 'StoreType', 'Assortment', 'CompetitionDistance',
       'CompetitionOpenSinceMonth', 'CompetitionOpenSinceYear', 'Promo2',
       'Promo2SinceWeek', 'Promo2SinceYear', 'PromoInterval', 'IsWeekend',
       'Year', 'Month', 'DayOfMonth', 'BeginMonth', 'MidMonth', 'EndMonth',
       'DayOfYear', 'IsWeekday'],
      dtype='object')

In [17]:
# lets check that all stores have same dates of holidays

train_data.StateHoliday.value_counts()

0    986159
a     20260
b      6690
c      4100
Name: StateHoliday, dtype: int64